In [1]:
import time
import random
import requests
import pandas as pd

VS = "usd"
DAYS = 90

BASE_LIST   = "https://api.coingecko.com/api/v3/coins/list"
BASE_CHART  = "https://api.coingecko.com/api/v3/coins/{id}/market_chart"

session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (compatible; stable-demo/1.0)"
})


def fetch_all_coins():
    """全銘柄リスト取得"""
    r = session.get(BASE_LIST, timeout=30)
    r.raise_for_status()
    coins = r.json()   # [{id, symbol, name}, ...]
    return coins


def fetch_market_chart(coin_id, days=90, vs="usd", max_retry=3):
    """指定コインの過去データ取得 (CoinGecko market_chart)"""
    url = BASE_CHART.format(id=coin_id)
    params = {"vs_currency": vs, "days": days}

    for attempt in range(max_retry):
        r = session.get(url, params=params, timeout=30)

        if r.status_code == 200:
            data = r.json()
            if "prices" not in data:
                raise RuntimeError(f"no 'prices' field for {coin_id}")
            df = pd.DataFrame(data["prices"], columns=["timestamp", "price"])
            df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
            return df

        # レートリミット
        if r.status_code == 429:
            wait_sec = 5 * (attempt + 1)
            print(f"[429] Too Many Requests. wait {wait_sec} sec... (attempt {attempt+1}/{max_retry})")
            time.sleep(wait_sec)
            continue

        # その他のエラー
        r.raise_for_status()

    raise RuntimeError(f"Failed after retries: {coin_id}")


def main():
    # 1) 全銘柄リスト取得
    all_coins = fetch_all_coins()
    print(f"total coins: {len(all_coins)}")

    # 2) ランダムに10銘柄を選択
    #   （あまり流動性のないコインも混ざります。上位だけにしたい場合はここでフィルタ）
    sample_size = 10
    coins_sampled = random.sample(all_coins, sample_size)

    for idx, coin in enumerate(coins_sampled, start=1):
        coin_id = coin["id"]
        symbol  = coin["symbol"]
        name    = coin["name"]

        print(f"\n[{idx}/{sample_size}] Fetching {name} ({symbol}) id={coin_id}")

        try:
            df = fetch_market_chart(coin_id, DAYS, VS)
        except Exception as e:
            print(f"  -> error: {e}")
            continue

        # 3) CSV に保存
        # ファイル名: rank_symbol_id_90d_usd.csv みたいな形
        safe_symbol = symbol.upper()
        safe_id = coin_id.replace("/", "_")
        filename = f"{idx:02d}_{safe_symbol}_{safe_id}_last{DAYS}d_{VS}.csv"

        df.to_csv(filename, index=False)
        print(f"  -> saved to {filename}")

        # 4) レートリミット回避のため少し待つ
        time.sleep(15)


if __name__ == "__main__":
    main()

total coins: 18770

[1/10] Fetching Wrapped SOPH (wsoph) id=wrapped-soph
  -> saved to 01_WSOPH_wrapped-soph_last90d_usd.csv

[2/10] Fetching tensorprox (sn91) id=tensorprox
  -> saved to 02_SN91_tensorprox_last90d_usd.csv

[3/10] Fetching LP 3pool Curve (3crv) id=lp-3pool-curve
  -> saved to 03_3CRV_lp-3pool-curve_last90d_usd.csv

[4/10] Fetching bullish (bullish) id=bullish
  -> saved to 04_BULLISH_bullish_last90d_usd.csv

[5/10] Fetching bePAY Finance (becoin) id=bepay
  -> saved to 05_BECOIN_bepay_last90d_usd.csv

[6/10] Fetching Stargate Bridged WETH (Fuse) (weth) id=stargate-bridged-weth-fuse
  -> saved to 06_WETH_stargate-bridged-weth-fuse_last90d_usd.csv

[7/10] Fetching Where Did The ETH Go? (Pulsechain) (wheth) id=where-did-the-eth-go-pulsechain
  -> saved to 07_WHETH_where-did-the-eth-go-pulsechain_last90d_usd.csv

[8/10] Fetching Monero (xmr) id=monero
  -> saved to 08_XMR_monero_last90d_usd.csv

[9/10] Fetching LayerZero Bridged Sei (sei) id=layerzero-bridged-sei
  -> save